# Description

According to the settings specified below, this notebook:
 1. reads all the data from one source (GTEx, recount2, etc) according to the gene selection method (`GENE_SELECTION_STRATEGY`),
 2. runs a quick performance test using the correlation coefficient specified (`CORRELATION_METHOD`), and
 3. computes the correlation matrix across all the genes using the correlation coefficient specified.

# Modules

In [1]:
import pandas as pd

from clustermatch import conf
from clustermatch.corr import spearman

# Settings

In [2]:
# we don't have gene subsets for recount2
# GENE_SELECTION_STRATEGY = "var_raw"

In [3]:
CORRELATION_METHOD = spearman

method_name = CORRELATION_METHOD.__name__
display(method_name)

'spearman'

In [4]:
PERFORMANCE_TEST_N_TOP_GENES = 500

# Paths

In [5]:
INPUT_FILE = conf.RECOUNT2["DATA_FILE"]
display(INPUT_FILE)

assert INPUT_FILE.exists()

PosixPath('/opt/data/data/recount2/recount_data_prep_PLIER.pkl')

In [6]:
OUTPUT_DIR = conf.RECOUNT2["SIMILARITY_MATRICES_DIR"]
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
display(OUTPUT_DIR)

PosixPath('/opt/data/results/recount2/similarity_matrices')

# Data loading

In [7]:
data = pd.read_pickle(INPUT_FILE)

In [8]:
data.shape

(6750, 37032)

In [9]:
data.head()

,SRP000599.SRR013549,SRP000599.SRR013550,SRP000599.SRR013551,SRP000599.SRR013552,SRP000599.SRR013553,SRP000599.SRR013554,SRP000599.SRR013555,SRP000599.SRR013556,SRP000599.SRR013557,SRP000599.SRR013558,...,SRP035599.SRR1139372,SRP035599.SRR1139393,SRP035599.SRR1139388,SRP035599.SRR1139378,SRP035599.SRR1139399,SRP035599.SRR1139386,SRP035599.SRR1139375,SRP035599.SRR1139382,SRP035599.SRR1139356,SRP035599.SRR1139370
GAS6,-0.312500,-0.312931,-0.312931,-0.312931,-0.312931,-0.308253,-0.312931,-0.312931,-0.312931,-0.312931,...,-0.301711,-0.305581,-0.303344,-0.297800,-0.307122,-0.285499,-0.309599,-0.300220,-0.297667,-0.310151
MMP14,-0.328279,-0.328279,-0.328279,-0.328279,-0.328279,-0.328279,-0.328279,-0.328279,-0.328279,-0.325140,...,-0.314587,-0.322952,-0.326439,-0.325994,-0.326272,-0.322523,-0.326375,-0.326339,-0.322127,-0.327438
DSP,-0.286319,-0.286859,-0.286859,-0.286859,-0.286859,-0.286859,-0.277195,-0.256862,-0.278790,-0.269701,...,-0.286859,-0.286859,-0.286745,-0.286688,-0.286725,-0.286529,-0.286859,-0.286671,-0.286859,-0.286740
MARCKSL1,-0.536646,-0.536646,-0.536646,-0.536646,-0.536646,-0.536646,-0.536646,-0.536646,-0.536646,-0.536646,...,0.807663,1.294564,1.527655,1.404788,1.047931,0.892119,1.507099,2.458255,2.919662,1.410846
SPARC,-0.370498,-0.370498,-0.369171,-0.370498,-0.370498,-0.370498,-0.370498,-0.370498,-0.370498,-0.370498,...,-0.345409,-0.310750,-0.348120,-0.356938,-0.355206,-0.366197,-0.351174,-0.363703,-0.350825,-0.360762


# Compute similarity

## Performance test

In [10]:
# select a subset of the genes
test_data = data.sample(n=PERFORMANCE_TEST_N_TOP_GENES, random_state=0)

In [11]:
test_data.shape

(500, 37032)

In [12]:
test_data.head()

,SRP000599.SRR013549,SRP000599.SRR013550,SRP000599.SRR013551,SRP000599.SRR013552,SRP000599.SRR013553,SRP000599.SRR013554,SRP000599.SRR013555,SRP000599.SRR013556,SRP000599.SRR013557,SRP000599.SRR013558,...,SRP035599.SRR1139372,SRP035599.SRR1139393,SRP035599.SRR1139388,SRP035599.SRR1139378,SRP035599.SRR1139399,SRP035599.SRR1139386,SRP035599.SRR1139375,SRP035599.SRR1139382,SRP035599.SRR1139356,SRP035599.SRR1139370
S1PR5,-0.266852,-0.266852,-0.266852,-0.266852,-0.266852,-0.266852,-0.266852,-0.266852,-0.266852,-0.266852,...,-0.266852,-0.266852,-0.261464,-0.262521,-0.263571,-0.232207,-0.263313,-0.263926,-0.266852,-0.266852
NDUFA12,-0.389020,-0.389020,-0.389020,-0.389020,-0.389020,-0.389020,-0.389020,-0.389020,-0.389020,-0.389020,...,0.264052,0.141801,-0.101283,0.016574,0.054545,0.109291,-0.080455,-0.039797,-0.171493,-0.039619
EXOSC10,-0.775256,-0.781191,-0.781191,-0.781191,-0.781191,-0.781191,-0.781191,-0.781191,-0.781191,-0.781191,...,-0.151619,-0.190753,-0.263083,0.006409,-0.090697,-0.196750,-0.039938,0.155149,0.076707,0.201055
ALOX15B,-0.104175,-0.104175,-0.104175,-0.104175,-0.104175,-0.104175,-0.104175,-0.104175,-0.104175,-0.104175,...,-0.102583,-0.104175,-0.104175,-0.090938,-0.104175,-0.091455,-0.097841,-0.097053,-0.098525,-0.102490
CACNB1,-0.412386,-0.412386,-0.412386,-0.412386,-0.412386,-0.412386,-0.412386,-0.412386,-0.412386,-0.412386,...,-0.002829,-0.176210,-0.124728,-0.034000,-0.170579,-0.134903,-0.050322,-0.160912,-0.034017,-0.001866


This is a quick performance test of the correlation measure. The following line (`_tmp = ...`) is the setup code, which is needed in case the correlation method was optimized using `numba` and needs to be compiled before performing the test.

In [13]:
_tmp = CORRELATION_METHOD(test_data.iloc[:3])

display(_tmp.shape)
display(_tmp)

(3, 3)

,S1PR5,NDUFA12,EXOSC10
S1PR5,1.000000,0.08518,0.273537
NDUFA12,0.085180,1.00000,0.416500
EXOSC10,0.273537,0.41650,1.000000


In [14]:
%timeit CORRELATION_METHOD(test_data)

5 s ± 4.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Run

In [15]:
# compute correlations
data_corrs = CORRELATION_METHOD(data)

In [16]:
display(data_corrs.shape)

assert data.shape[0] == data_corrs.shape[0]

(6750, 6750)

In [17]:
data_corrs.head()

,GAS6,MMP14,DSP,MARCKSL1,SPARC,CTSD,EPAS1,PALLD,PHC2,LGALS3BP,...,LDHB,LDHC,ACAP2,ACAP3,CFL2,CFL1,NFIB,PLEKHG6,GNGT2,SERPINH1
GAS6,1.000000,0.767277,0.530435,0.376058,0.492968,0.652806,0.694218,0.598491,0.599584,0.595290,...,0.096758,0.319135,0.243757,0.669781,0.433053,0.397045,0.298712,0.484416,0.357766,0.689587
MMP14,0.767277,1.000000,0.490369,0.372943,0.593585,0.608753,0.652577,0.566899,0.596954,0.591671,...,0.108111,0.328656,0.295322,0.573781,0.372953,0.388056,0.232660,0.498833,0.371384,0.707476
DSP,0.530435,0.490369,1.000000,0.309156,0.250510,0.378827,0.570500,0.603818,0.261268,0.483510,...,0.102505,0.322113,0.181120,0.336312,0.389993,0.143608,0.396473,0.567024,0.022467,0.615777
MARCKSL1,0.376058,0.372943,0.309156,1.000000,0.251116,0.380241,0.254586,0.286491,0.441862,0.339611,...,0.386577,0.314713,0.302932,0.548047,0.313833,0.454887,0.228858,0.352535,0.312589,0.478413
SPARC,0.492968,0.593585,0.250510,0.251116,1.000000,0.299277,0.404538,0.481395,0.353212,0.426300,...,0.259254,0.219185,0.128572,0.276951,0.441842,0.182518,0.339385,0.180845,0.240672,0.503508


In [18]:
output_filename = OUTPUT_DIR / f"{INPUT_FILE.stem}-{method_name}.pkl"
display(output_filename)

PosixPath('/opt/data/results/recount2/similarity_matrices/recount_data_prep_PLIER-spearman.pkl')

In [19]:
# save
data_corrs.to_pickle(output_filename)